In [12]:
# importing the libraries
import pandas as pd
import numpy as np
import math
# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline
# for creating validation set
from sklearn.model_selection import train_test_split
# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm
# PyTorch libraries and modules
import torch
import torch.nn as nn
import torchvision
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout


from torch.optim import Adam, SGD
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

import time

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay


# self write tools
from tools import LABEL_ENCODER, transforms_train, transforms_test
from models import CNN, RNN, NN
from dataset import CnnDataset, TestDataset, CREATE_CNN_TRAIN_ITERATOR, CREATE_CNN_VALID_ITERATOR, words, TEST_DATASET

# Check GPU

In [13]:
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
torch.backends.cudnn.benchmark = True
print("Using {}: {}".format(device, torch.cuda.get_device_name(0)))

Using cuda: GeForce RTX 3080


In [14]:
class MyEnsemble(nn.Module):
    def __init__(self, _cnn, _rnn, _nn):
        super(MyEnsemble, self).__init__()
        self.cnn = _cnn
        self.rnn = _rnn
        self.nn = _nn
        self.classifier2 = nn.Linear(81, 200)
        self.classifier1 = nn.Linear(200, 100)
        self.classifier = nn.Linear(100, 27)
        
    def forward(self, image, description, l, features):
        x1 = self.cnn(image)
        x2 = self.rnn(description, l)
        x3 = self.nn(features)


        x = torch.cat((x1, x2, x3), dim=1)
        x = self.classifier2(F.relu(x))
        x = self.classifier1(F.relu(x))
        x = self.classifier(F.relu(x))
        return x

# Global Variables

In [15]:

BATCH_SIZE = 256
EPOCHS = 50





# 2. Model

In [16]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (image, (description, l), features, y) in iterator:

        description = description.to(device, dtype=torch.long)
        # l = l.to(device, dtype=torch.long)
        l = l.long()
        
        image = image.to(device)
        features = features.to(device)
        y = y.to(device, dtype=torch.int64)
        
        optimizer.zero_grad()
                
        # y_pred = model(image)
        y_pred = model(image, description, l, features)
        
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate_cnn(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():

        for (image, (description, l), features, y) in iterator:

            description = description.to(device, dtype=torch.long)
            # l = l.to(device, dtype=torch.long)
            l = l.long()
            
            image = image.to(device)
            features = features.to(device)
        
            y = y.to(device, dtype=torch.int64)
                    
            # y_pred = model(image)
            y_pred = model(image, description, l, features)
            
            loss = criterion(y_pred, y)
            
            acc = calculate_accuracy(y_pred, y)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)



In [17]:
cnn_train_iterator = CREATE_CNN_TRAIN_ITERATOR(transforms_train, BATCH_SIZE)
cnn_valid_iterator = CREATE_CNN_VALID_ITERATOR(BATCH_SIZE)

In [18]:
criterion = CrossEntropyLoss()
criterion = criterion.cuda()


cnn_model = CNN()
cnn_model = cnn_model.cuda()
cnn_model.load_state_dict(torch.load('output/cnn-model.pt'))


rnn_model = RNN()
rnn_model = rnn_model.cuda()
rnn_model.load_state_dict(torch.load('output/rnn-model.pt'))
# test_loss, test_acc = evaluate_cnn(rnn_model, cnn_valid_iterator, criterion, device)
# print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

nn_model = NN(4, 15, 27)
nn_model = nn_model.cuda()
nn_model.load_state_dict(torch.load('output/linear-model.pt'))

ensamble_model = MyEnsemble(cnn_model, rnn_model, nn_model)
ensamble_model = ensamble_model.cuda()

# model = cnn_model
model = ensamble_model

optimizer = Adam(model.parameters(), lr=0.0001)









In [19]:

model.load_state_dict(torch.load('output/combine-model.pt'))
test_loss, test_acc = evaluate_cnn(model, cnn_valid_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


train_accuracy_list = []
train_loss_list = []
valid_acc_list = []
valid_loss_list = []
# best_valid_loss = float('inf')
best_valid_loss = test_loss

for epoch in range(EPOCHS):

    start_time = time.monotonic()

    # train
    train_loss, train_acc = train(model, cnn_train_iterator, optimizer, criterion, device)
    
    # valid
    valid_loss, valid_acc = evaluate_cnn(model, cnn_valid_iterator, criterion, device)

    # save best model
    if valid_loss < best_valid_loss:
        print('Best Epoch:', epoch)
        print('Best Accuracy:', valid_acc)
        print('Best Loss:', valid_loss)
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'output/combine-model.pt')

    # Track the accuracy
    train_accuracy_list.append(train_acc)
    train_loss_list.append(train_loss)
    valid_acc_list.append(valid_acc)
    valid_loss_list.append(valid_loss)
        
    # print epoch info
    end_time = time.monotonic()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
        




Test Loss: 0.124 | Test Acc: 97.51%
Epoch: 01 | Epoch Time: 0m 31s
	Train Loss: 0.006 | Train Acc: 99.80%
	 Val. Loss: 0.141 |  Val. Acc: 97.35%
Epoch: 02 | Epoch Time: 0m 31s
	Train Loss: 0.005 | Train Acc: 99.85%
	 Val. Loss: 0.139 |  Val. Acc: 97.30%
Epoch: 03 | Epoch Time: 0m 31s
	Train Loss: 0.005 | Train Acc: 99.83%
	 Val. Loss: 0.142 |  Val. Acc: 97.44%
Epoch: 04 | Epoch Time: 0m 31s
	Train Loss: 0.006 | Train Acc: 99.87%
	 Val. Loss: 0.132 |  Val. Acc: 97.48%
Epoch: 05 | Epoch Time: 0m 31s
	Train Loss: 0.004 | Train Acc: 99.93%
	 Val. Loss: 0.150 |  Val. Acc: 97.19%
Epoch: 06 | Epoch Time: 0m 31s
	Train Loss: 0.003 | Train Acc: 99.91%
	 Val. Loss: 0.144 |  Val. Acc: 97.29%
Epoch: 07 | Epoch Time: 0m 31s
	Train Loss: 0.003 | Train Acc: 99.90%
	 Val. Loss: 0.135 |  Val. Acc: 97.37%
Epoch: 08 | Epoch Time: 0m 31s
	Train Loss: 0.004 | Train Acc: 99.91%
	 Val. Loss: 0.152 |  Val. Acc: 97.19%
Epoch: 09 | Epoch Time: 0m 31s
	Train Loss: 0.006 | Train Acc: 99.84%
	 Val. Loss: 0.163 |  

In [28]:
model.load_state_dict(torch.load('output/combine-model.pt'))
test_loss, test_acc = evaluate_cnn(model, cnn_valid_iterator, criterion, device)

print(f'Test Loss: {test_loss:.5f} | Test Acc: {test_acc*100:.5f}%')

Test Loss: 0.12453 | Test Acc: 97.50280%


In [24]:
def predict(model):
    dataloader = DataLoader(TEST_DATASET, batch_size=1024)

    df = pd.DataFrame()
    model.eval()
        
    with torch.no_grad():

        for (images, (description, l), ids, features) in tqdm(dataloader):

            description = description.to(device, dtype=torch.long)
                # l = l.to(device, dtype=torch.long)
            l = l.long()
                
            images = images.to(device)
            features = features.to(device)
            
                        
            y_pred = model(images, description, l, features)
            

            top_pred = y_pred.argmax(1).cpu()
            labels = LABEL_ENCODER.inverse_transform(top_pred)

            temp = pd.DataFrame()
            temp['id'] = ids
            temp['category'] = labels
            

            df = df.append(temp)


    print(df)
    df.to_csv('output/prediction.csv', index=False)


In [29]:
predict(model)

100%|██████████| 22/22 [00:11<00:00,  1.90it/s]        id   category
0    26266  Innerwear
1    22134    Topwear
2    28358      Belts
3    15554      Shoes
4    53408  Innerwear
..     ...        ...
119  39737    Eyewear
120  57477      Dress
121  22312    Topwear
122  54105       Lips
123  14080       Bags

[21628 rows x 2 columns]

